In [6]:
import os
os.chdir('../')


'c:\\Users\\Joe\\Desktop\\Term-2\\Data_Science\\DS_CW_2'

In [8]:
from hltv_api import main as hltv
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [10]:
## gets list of recent (last 6 months) event id's for top tier tournaments (Defined as International LAN's with prize pool greater or equal to 100000$)

events_page = hltv.get_parsed_page('https://www.hltv.org/events/archive?startDate=2023-10-24&endDate=2024-04-24&eventType=INTLLAN&eventType=LOCALLAN&prizeMin=100000&prizeMax=2000000')


<!DOCTYPE html>
<html data-bc-content-allowed="false" data-client-country-iso="gb" data-impression-tracking-endpoint="https://ncuxt.hltv.org" lang="en"><!--Active rewriter version: 1-->
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" id="metaViewport" name="viewport"/>
<meta content="1004164229" property="fb:admins"/>
<meta content="249997999009" property="fb:pages"/>
<meta content="1460388157605817" property="fb:app_id"/>
<meta content="DcypRFLQvgYQL5Acx7feoGWbblSsmKv6HpPI7mM_1uw" name="google-site-verification"/>
<meta content="5OtriQwtVytxCNrtDRVt0gtjmAN81BJ9Z6ZLLCXEYwQ" name="google-site-verification"/>
<meta content="00ed04feb3ede037" name="yandex-verification"/>
<link href="/img/static/favicon/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/img/static/favicon/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/img/static/favicon/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<li

In [15]:
event_ids = []
for event in events_page.find_all('a', attrs={'class': 'a-reset small-event standard-box'}):
    event_id = event.get('href').split('/')[2]
    event_ids.append(event_id)


['7742', '7437', '7435', '7552', '6869', '6976', '6923', '7499', '7250', '6868', '6975', '7527', '7365']


In [16]:
## make dataframes per event
df_per_event = []
for id in event_ids:
    data = hltv.get_results_by_event(event_id) ## get all matches and outcome data for tournament
    df = pd.DataFrame(data)

    ## This gets further matchd ata for each match
    match_ids = df['match-id']
    match_stats = []
    for match_id in match_ids:
        match_stats.append(hltv.get_match_stats(match_id))

    df_match_stats = pd.DataFrame(match_stats)
    df = pd.merge(df, df_match_stats, on='match-id', how = 'inner') ## combine dataframes
    
    event_team_rankings = hltv.get_event_team_rankings(event_id) ## gets team rankings at time of the event
    team_rankings_df = pd.DataFrame(list(event_team_rankings.items()), columns=['Team', 'Ranking'])

    ## Add team rankings to dataframe
    df = pd.merge(df, team_rankings_df, left_on='team1', right_on='Team', how = 'left')
    df = df.rename(columns={'Ranking': 'team1_Ranking'}).drop(columns=['Team'])
    df = pd.merge(df, team_rankings_df, left_on='team2', right_on='Team', how = 'left')
    df = df.rename(columns={'Ranking': 'team2_Ranking'}).drop(columns=['Team'])

    df_per_event.append(df)



In [19]:
df = pd.concat(df_per_event)